In [4]:
import os
import time
import random
import pandas as pd
import cloudscraper
from bs4 import BeautifulSoup, Comment
import requests

# Initialize cloudscraper session
scraper = cloudscraper.create_scraper(
    browser={"browser": "chrome", "platform": "windows", "desktop": True}
)
BASE_URL = "https://fbref.com"

# Top-5 leagues: name -> comp_id
LEAGUES = {
    'Premier League': 9,
    'La Liga':       12,
    'Bundesliga':    20,
    'Serie A':       11,
    'Ligue 1':       13,
}
# Stats categories: key -> URL segment ('' means overall)
CATEGORIES = {
    'standard':    '',
    'shooting':    'shooting',
    'passing':     'passing',
    'gca':         'gca',
    'defense':     'defense',
    'possession':  'possession',
    'misc':        'misc',
}

# Request throttling and retry settings
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
    '(KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4) AppleWebKit/605.1.15 '
    '(KHTML, like Gecko) Version/16.5 Safari/605.1.15',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 '
    '(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:117.0) Gecko/20100101 Firefox/117.0',
]
PROXIES = []  # e.g. ['http://proxy1:port', ...]
DELAY_MIN = 3
DELAY_MAX = 6
MAX_RETRIES = 5
BACKOFF_FACTOR = 2


def get_with_retries(url, referer=None):
    """
    Fetch URL with retries on 429/network errors. Exponential backoff + jitter.
    """
    last_err = None
    for attempt in range(1, MAX_RETRIES + 1):
        headers = {'User-Agent': random.choice(USER_AGENTS)}
        if referer:
            headers['Referer'] = referer
        proxies = None
        if PROXIES:
            proxy = random.choice(PROXIES)
            proxies = {'http': proxy, 'https': proxy}
        try:
            resp = scraper.get(url, headers=headers, proxies=proxies)
            if resp.status_code == 429:
                raise requests.exceptions.HTTPError('429 Too Many Requests')
            resp.raise_for_status()
            return resp
        except Exception as e:
            last_err = e
            if attempt == MAX_RETRIES:
                print(f"[Error] {url} failed after {MAX_RETRIES} attempts: {e}")
                raise
            wait = (BACKOFF_FACTOR ** (attempt - 1)) + random.uniform(0, 1)
            print(f"[Retry {attempt}] {url} error: {e}. Sleeping {wait:.1f}s...")
            time.sleep(wait)
    raise last_err


def scrape_league_category(league_name: str, comp_id: int, category_key: str) -> pd.DataFrame:
    """
    Scrape player stats for one league and category.
    Returns DataFrame with extra columns: League, Category.
    """
    segment = CATEGORIES[category_key]
    path = f"{segment}/" if segment else ''
    # Build URL
    url = f"{BASE_URL}/en/comps/{comp_id}/{path}{league_name.replace(' ', '-')}-Stats"
    print(f"Fetching {league_name} [{category_key}] stats...")
    time.sleep(random.uniform(DELAY_MIN, DELAY_MAX))
    resp = get_with_retries(url)
    soup = BeautifulSoup(resp.text, 'lxml')

    # Target player stats table in commented block
    block_id = f"div_stats_{segment or 'standard'}"
    comment = soup.find(string=lambda t: isinstance(t, Comment) and block_id in t)
    if comment:
        tbl_soup = BeautifulSoup(comment, 'lxml')
        table = tbl_soup.find('table')
    else:
        # fallback: first player stats table by id prefix
        table = soup.find('table', id=lambda x: x and x.startswith('stats_'))

    if table is None or not table.tbody:
        print(f"[Warning] No player table for {league_name} {category_key}")
        return pd.DataFrame()

    df = pd.read_html(str(table))[0]
    # Label league and category
    df['League'] = league_name
    df['Category'] = category_key
    return df


def scrape_all_categories():
    """
    Scrape all categories across all leagues, saving each category into its own DataFrame and CSV.
    Returns dict: category_key -> DataFrame
    """
    os.makedirs('data', exist_ok=True)
    category_dfs = {}
    for category_key in CATEGORIES.keys():
        frames = []
        for league, comp_id in LEAGUES.items():
            try:
                df = scrape_league_category(league, comp_id, category_key)
                if not df.empty:
                    frames.append(df)
            except Exception as e:
                print(f"Error scraping {league} {category_key}: {e}")
        # Combine leagues for this category
        if frames:
            cat_df = pd.concat(frames, ignore_index=True)
        else:
            cat_df = pd.DataFrame()
        # Save to CSV
        cat_df.to_csv(f"data/{category_key}_player_stats.csv", index=False)
        category_dfs[category_key] = cat_df
        print(f"Saved {category_key} stats: {cat_df.shape[0]} rows")
    return category_dfs


def main():
    dfs = scrape_all_categories()
    # Example access: dfs['defense'] for combined defensive stats
    return dfs

if __name__ == '__main__':
    all_category_dfs = main()
    # Print head of one category
    print(all_category_dfs['standard'].head())


Fetching Premier League [standard] stats...


/var/folders/mp/l7pc43ln25l8vb_t3fc17rlm0000gn/T/ipykernel_61658/2025400879.py:109: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


Fetching La Liga [standard] stats...


/var/folders/mp/l7pc43ln25l8vb_t3fc17rlm0000gn/T/ipykernel_61658/2025400879.py:109: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


Fetching Bundesliga [standard] stats...


...

/var/folders/mp/l7pc43ln25l8vb_t3fc17rlm0000gn/T/ipykernel_61658/2025400879.py:109: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


Fetching Serie A [misc] stats...


/var/folders/mp/l7pc43ln25l8vb_t3fc17rlm0000gn/T/ipykernel_61658/2025400879.py:109: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


Fetching Ligue 1 [misc] stats...
Saved misc stats: 2966 rows
  Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
               Squad               # Pl                Age               Poss   
0            Arsenal                 25               25.8               56.9   
1        Aston Villa                 28               27.0               50.5   
2        Bournemouth                 29               25.1               48.5   
3          Brentford                 28               25.8               47.9   
4           Brighton                 32               24.8               52.3   

  Playing Time                    Performance      ... Per 90 Minutes        \
            MP Starts   Min   90s         Gls Ast  ...            G+A  G-PK   
0           38    418  3420  38.0          67  55  ...           3.21  1.71   
1           38    418  3420  38.0          56  45  ...           2.66  1.39   
2           38    418  3420  38.0          57  41  ... 

/var/folders/mp/l7pc43ln25l8vb_t3fc17rlm0000gn/T/ipykernel_61658/2025400879.py:109: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


In [29]:
gca_df.head()

,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,SCA,SCA.1,...,GCA.1,GCA Types,GCA Types.1,GCA Types.2,GCA Types.3,GCA Types.4,GCA Types.5,Unnamed: 24_level_0,League,Category
0,Rk,Player,Nation,Pos,Squad,Age,Born,90s,SCA,SCA90,...,GCA90,PassLive,PassDead,TO,Sh,Fld,Def,Matches,NaN,NaN
1,1,Max Aarons,eng ENG,DF,Bournemouth,24,2000,1.0,2,2.09,...,0.00,0,0,0,0,0,0,Matches,Premier League,gca
2,2,Joshua Acheampong,eng ENG,DF,Chelsea,18,2006,1.9,2,1.06,...,0.00,0,0,0,0,0,0,Matches,Premier League,gca
3,3,Tyler Adams,us USA,MF,Bournemouth,25,1999,21.8,41,1.88,...,0.18,3,0,0,0,0,1,Matches,Premier League,gca
4,4,Tosin Adarabioyo,eng ENG,DF,Chelsea,26,1997,15.7,15,0.96,...,0.06,0,0,0,1,0,0,Matches,Premier League,gca


In [14]:
gca_df = pd.read_csv('data/gca_player_stats.csv')
defense_df = pd.read_csv('data/defense_player_stats.csv')
misc_df = pd.read_csv('data/misc_player_stats.csv')
passing_df = pd.read_csv('data/passing_player_stats.csv')
possession_df = pd.read_csv('data/possession_player_stats.csv')
shooting_df = pd.read_csv('data/shooting_player_stats.csv')


In [30]:
overall_df = pd.DataFrame()

overall_df['Unnamed: 1_level_0'] = overall_df['Unnamed: 1_level_0']
overall_df['Unnamed: 4_level_0'] = overall_df['Unnamed: 4_level_0']

for df in [gca_df, defense_df, misc_df, passing_df, possession_df, shooting_df]:
    overall_df = pd.merge(overall_df, df, on=['Unnamed: 1_level_0', 'Unnamed: 4_level_0'], how='left')
    overall_df.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'], inplace=True)

overall_df.head(30)

KeyError: 'Unnamed: 1_level_0'

In [36]:
all_dfs = [gca_df, defense_df, misc_df, passing_df, possession_df, shooting_df]

for df in all_dfs:
    for column in df.columns:
        column = df[column].iloc[0]
    df.drop(index=0, inplace=True)

In [37]:
gca_df.head()

,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,SCA,SCA.1,...,GCA.1,GCA Types,GCA Types.1,GCA Types.2,GCA Types.3,GCA Types.4,GCA Types.5,Unnamed: 24_level_0,League,Category
1,1,Max Aarons,eng ENG,DF,Bournemouth,24,2000,1.0,2,2.09,...,0.00,0,0,0,0,0,0,Matches,Premier League,gca
2,2,Joshua Acheampong,eng ENG,DF,Chelsea,18,2006,1.9,2,1.06,...,0.00,0,0,0,0,0,0,Matches,Premier League,gca
3,3,Tyler Adams,us USA,MF,Bournemouth,25,1999,21.8,41,1.88,...,0.18,3,0,0,0,0,1,Matches,Premier League,gca
4,4,Tosin Adarabioyo,eng ENG,DF,Chelsea,26,1997,15.7,15,0.96,...,0.06,0,0,0,1,0,0,Matches,Premier League,gca
5,5,Simon Adingra,ci CIV,"FW,MF",Brighton,22,2002,12.2,47,3.86,...,0.57,5,0,1,1,0,0,Matches,Premier League,gca
